In [1]:
import torch
import gym
from gym import envs
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from torch.autograd import Variable

In [2]:
lenobs = 100800
class ActorCritic(nn.Module):
    def __init__(self):
        super(ActorCritic, self).__init__()
        self.l1 = nn.Linear(lenobs,25)
        self.l2 = nn.Linear(25,50)
        self.actor_lin1 = nn.Linear(50,2)
        self.l3 = nn.Linear(50,25)
        self.critic_lin1 = nn.Linear(25,1)

    def forward(self,x):
        x = F.normalize(x,dim=0)
        y = F.relu(self.l1(x))
        y = F.relu(self.l2(y))
        actor = F.softmax(self.actor_lin1(y),dim=0)
#         actor = F.log_softmax(self.actor_lin1(y),dim=0)
        c = F.relu(self.l3(y.detach()))
        critic = torch.tanh(self.critic_lin1(c))
        return actor, critic

In [4]:
env = gym.make('PongNoFrameskip-v0')
moveMapping = {
    0:2,
    1:3
}

model = ActorCritic()
optimizer = optim.Adam(lr=1e-4,params=model.parameters())

'''
loss = Variable(loss, requires_grad = True)
actor_loss = Variable(actor_loss, requires_grad = True)
critic_loss = Variable(critic_loss, requires_grad = True)
'''

for i_episode in range(1):
#     reward = 0.0
    values = []
    rewards = []
    logprobs = []
    observation = env.reset()
    print('---------------')
    done = False
    t = 0
    while done == False:
        t+=1
#         print(t)
        pobservation = torch.from_numpy(observation)
        flattened_pobservation = pobservation.view(-1).float()
        policy, value = model(flattened_pobservation)
#         print(policy)
        values.append(value)
        action = np.random.choice(np.array([0,1]), p = policy.view(2,).data.numpy())
        logprobs.append(policy.view(-1)[action])
#         print('Action: {}'.format('right' if action==2 else 'left'))
        observation, reward, done, log = env.step(moveMapping[action])
        rewards.append(reward)
#         print('{}.Reward: {}'.format(t,reward))
#         print('---')
        if done:
            print(done)
            print('Episode:{} State:{} Reward:{}'.format(i_episode,t,reward))
            print("Episode finished after {} timesteps".format(t+1))    
            break
        
      
    # Reversing because earlier actions have greater value
    torch_values = torch.tensor(values, requires_grad = True).view(-1).flip(0)
    torch_rewards = torch.tensor(rewards, requires_grad = True).flip(0)
    torch_logprobs = torch.tensor(logprobs, requires_grad = True).flip(0)
    
    
    returns = []
    gamma = 0.95
    clc = 0.1
    ret = torch.tensor([0])
    for r in torch_rewards:
        ret = r + gamma*ret
        returns.append(ret)
    returns = torch.tensor(returns, requires_grad = True)
    returns = F.normalize(returns,dim=0)
    actor_loss = -1*torch_logprobs * (returns - torch_values)
    critic_loss = torch.pow(torch_values - returns,2)
    loss = actor_loss.sum() + clc*critic_loss.sum()
    print('Loss: {}'.format(loss))
#     print('Starting Backpropagation')
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
#     print('Completed Backpropagation')


---------------
True
Episode:0 State:3056 Reward:-1.0
Episode finished after 3057 timesteps
Loss: -14.934562683105469


In [3]:
import torch
from multiprocessing import Pool

In [ ]:
import torch
import torch.nn as nn



# def f(n):
#     return n+2

# p = Pool()

# t = torch.ones(10,5,10)


# # a = time.time()
# result = p.map(f, t) #This alone will divide the work among all available cores equally
# p.close()
# p.join()
# # b = time.time() - a
# # print('Time taken: {}'.format(b))
# # print(result)


In [147]:
import concurrent.futures

In [ ]:
import multiprocessing as mp

criterion = nn.CrossEntropyLoss()
target = torch.Tensor([1.]).long

def updateParams(mod):
    optim.zero_grad()
    loss = criterion(mod(x), target)
    loss.backward()
    optim.step()
    
x = torch.Tensor([1.,2.,3,])
optim = optimizer.Adam(lr = 1e-1, params = mod.parameters())
mod = nn.Sequential(
    nn.Linear(3,2),
    nn.Softmax(dim = 0)
)

for i in mod.parameters():
    print(i)
    
mod.share_memory()

processes = []
for rank in range(1):
    p = mp.Process(target=updateParams, args=(mod,))
    p.start()
    processes.append(p)
for p in processes:
    p.join()

for i in mod.parameters():
    print(i)   

In [ ]:

criterion = nn.CrossEntropyLoss()
x = torch.Tensor([1.,2.,3,]).reshape(1,3)
print(x.shape)

mod = nn.Linear(3,2)
optim = optimizer.SGD(lr = 5, params = mod.parameters())
for i in mod.parameters():
    print(i)
print()
optim.zero_grad()
pred = nn.Softmax(dim=0)(mod(x)).reshape(1,2)
print(pred.shape)
target = torch.Tensor([0])
print(target.shape)
loss = criterion(pred, target.long())
loss.backward()
optim.step()
for i in mod.parameters():
    print(i)
print()
pred = nn.Softmax(dim=0)(mod(x)).reshape(1,2)
optim.zero_grad()
loss = criterion(pred, target.long())
loss.backward()
optim.step()
for i in mod.parameters():
    print(i)
print()